In [125]:
from gurobipy import Model, GRB, quicksum
"""Model:
The main Gurobi optimization object. Holds all decision variables,
constraints, and the objective function. Provides methods such as
addVar(), addConstr(), setObjective(), and optimize().
"""
"""GRB:
A namespace containing Gurobi constants, such as variable types
(GRB.BINARY, GRB.CONTINUOUS) and objective senses (GRB.MINIMIZE,
GRB.MAXIMIZE).
"""
"""quicksum:
Gurobi’s efficient linear-expression builder. Faster and more memory-
efficient than Python’s built-in sum() when summing decision variables.
 """


import math

**MASTER**

In [126]:


def _get_b_node(b_node, i, k):
    """
    Helper to read b_{ik}. Supports:
      - dict[(i,k)] -> value
      - list-of-lists b_node[i][k]
    """
    if isinstance(b_node, dict):
        return float(b_node.get((i, k), 0.0))
    return float(b_node[i][k])

def _get_b_supply(b_supply, k):
    """
    Helper to read b_k. Supports:
      - dict[k] -> value
      - list/tuple b_supply[k]
    """
    if isinstance(b_supply, dict):
        return float(b_supply[k])
    return float(b_supply[k])

def create_master_problem(
    V,
    E,
    P_list,
    R,
    b_node,
    b_supply,
    M_prec=1e6,
    M_flow=1e6,
    gurobi_output=False,
):
    """
    Build the initial master problem for a given scenario set P_list.

    V       : list of nodes (0 is source, V[-1] is sink n+1)
    E       : list of fixed precedence arcs (i,j) -> x_ij = 1
    P_list  : list of duration vectors p^h; each p^h has len(p^h) == len(V)
    R       : list of resource indices (e.g. [0,1,...])
    b_node  : node by resource balances b_{ik}, as dict[(i,k)] or 2D list [i][k]
    b_supply: per-resource supply b_k, as dict[k] or list
    M_prec  : big-M for precedence constraints
    M_flow  : big-M for capacity constraints 0 <= f_ijk <= M_flow * x_ij

    Returns:
        m    : Gurobi model
        x    : x[i,j] binary policy arcs
        S    : dict of S[(h,i)] variables
        rho  : robust makespan variable
        f    : f[i,j,k] flow variables
    """
    m = gp.Model("master_rcpsp")
    if not gurobi_output:
        m.Params.OutputFlag = 0

    source = V[0]
    sink = V[-1]
    H = len(P_list)        # number of scenarios
    nV = len(V)

    # --- Decision variables ---

    # x_ij ∈ {0,1}
    x = m.addVars(V, V, vtype=GRB.BINARY, name="x")

    # S^h_i ≥ 0  (we store them in a dict S[(h,i)])
    S = {}
    for h, p in enumerate(P_list):
        for i in V:
            S[(h, i)] = m.addVar(lb=0.0, name=f"S[{h},{i}]")

    # ρ ≥ 0
    rho = m.addVar(lb=0.0, name="rho")

    # f_ijk ≥ 0
    f = m.addVars(V, V, R, lb=0.0, name="f")

    # --- Objective ---
    # min ρ
    m.setObjective(rho, GRB.MINIMIZE)

    # --- Constraints ---

    # (eq. rho_ge_S)  ρ ≥ S^h_{sink}  for all scenarios h
    for h in range(H):
        m.addConstr(rho >= S[(h, sink)], name=f"rho_lb[{h}]")

    # (eq. prec_scen)  S^h_j ≥ S^h_i + p^h_i - M(1 - x_ij)
    for h, p in enumerate(P_list):
        for i in V:
            p_hi = float(p[i])
            for j in V:
                if i == j:
                    continue
                m.addConstr(
                    S[(h, j)] >= S[(h, i)] + p_hi - M_prec * (1 - x[i, j]),
                    name=f"prec[{i},{j},{h}]"
                )

    # (eq. nonneg_S) is enforced by lb=0.0 on S^h_i

    # (eq. f_nonneg) is enforced by lb=0.0 on f_ijk

    # (eq. flow_conserv)  internal nodes: sum_j f_{j i k} = sum_j f_{i j k} = b_{ik}
    for k in R:
        for i in V:
            if i in (source, sink):
                continue
            inflow = gp.quicksum(f[j, i, k] for j in V if j != i)
            outflow = gp.quicksum(f[i, j, k] for j in V if j != i)
            b_ik = _get_b_node(b_node, i, k)
            m.addConstr(inflow == b_ik, name=f"flow_in[{i},{k}]")
            m.addConstr(outflow == b_ik, name=f"flow_out[{i},{k}]")

    # (eq. flow_supply)  source/sink: sum_j f_{0 j k} = sum_j f_{j,n+1,k} = b_k
    for k in R:
        b_k = _get_b_supply(b_supply, k)
        m.addConstr(
            gp.quicksum(f[source, j, k] for j in V if j != source) == b_k,
            name=f"supply_source[{k}]"
        )
        m.addConstr(
            gp.quicksum(f[j, sink, k] for j in V if j != sink) == b_k,
            name=f"supply_sink[{k}]"
        )

    # (eq. cap_flow)  0 ≤ f_{ijk} ≤ M * x_ij
    for k in R:
        for i in V:
            for j in V:
                if i == j:
                    # if you want, you can force no self-flow:
                    m.addConstr(f[i, j, k] == 0.0, name=f"no_self_flow[{i},{k}]")
                else:
                    m.addConstr(
                        f[i, j, k] <= M_flow * x[i, j],
                        name=f"cap[{i},{j},{k}]"
                    )

    # (eq. x_fixed)  x_{ij} = 1 for (i,j) in E
    for (i, j) in E:
        m.addConstr(x[i, j] == 1, name=f"x_fixed[{i},{j}]")

    # (eq. x_bin) is enforced by vtype=GRB.BINARY

    m.update()
    return m, x, S, rho, f



def add_scenario_to_master(
    m,
    x,
    S,
    rho,
    p_vec,
    h_new,
    V,
    M_prec=1e6,
):
    """
    Add a new scenario h_new with durations p_vec to an existing master model.

    m     : existing Gurobi model
    x     : x[i,j] binary vars (shared across scenarios)
    S     : dict of S[(h,i)] vars; this function will extend it
    rho   : robust makespan var
    p_vec : list/array of durations p^{h_new}_i (len == len(V))
    h_new : integer index of the new scenario (e.g. current number of scenarios)
    V     : list of nodes
    M_prec: big-M for precedence constraints
    """
    sink = V[-1]

    # Create S^{h_new}_i ≥ 0 for all i
    for i in V:
        S[(h_new, i)] = m.addVar(lb=0.0, name=f"S[{h_new},{i}]")

    # Add constraint ρ ≥ S^{h_new}_{sink}
    m.addConstr(rho >= S[(h_new, sink)], name=f"rho_lb[{h_new}]")

    # Add precedence constraints for scenario h_new:
    # S^{h_new}_j ≥ S^{h_new}_i + p^{h_new}_i - M*(1 - x_ij)
    for i in V:
        p_hi = float(p_vec[i])
        for j in V:
            if i == j:
                continue
            m.addConstr(
                S[(h_new, j)] >= S[(h_new, i)] + p_hi - M_prec * (1 - x[i, j]),
                name=f"prec[{i},{j},{h_new}]"
            )

    m.update()



def extract_policy_edges_from_x(x, V, tol=0.5):
    """
    Extract the current earliest-start policy X^q from the Gurobi variables x[i,j].

    Parameters
    ----------
    x   : Gurobi tupledict of binary vars x[i,j]
    V   : list of nodes (e.g. [0,1,...,n])
    tol : threshold above which x[i,j] is considered '1'

    Returns
    -------
    policy_edges : list of (i,j) such that x[i,j] ≈ 1
    """
    policy_edges = []
    for i in V:
        for j in V:
            if i == j:
                continue
            val = x[i, j].X
            if val is not None and val > tol:
                policy_edges.append((i, j))
    return policy_edges


**Adversary**

In [127]:
def build_pi_set(hat_p):
    """
    hat_p: list of deviations hat_p[i]
    returns sorted list Pi = {0} ∪ {hat_p_i}
    """
    values = {0.0}
    for v in hat_p:
        values.add(float(v))
    return sorted(values)

def compute_durations_for_pi(pi, bar_p, hat_p):
    """
    bar_p, hat_p: lists of same length
    returns list p_pi[i] = bar_p[i] + max(hat_p[i] - pi, 0)
    """
    n = len(bar_p)
    p_pi = [0.0] * n
    for i in range(n):
        p_pi[i] = float(bar_p[i]) + max(float(hat_p[i]) - pi, 0.0)
    return p_pi


# ===============================================================
#   Correct Robust Adversary for BS-budget RCPSP
#   (handles precedence arcs separately from policy arcs)
# ===============================================================


def solve_nominal_with_gurobi_simple(bar_p, edges_prec, edges_policy, source, sink, gurobi_output=False):
    """
    Compute longest path where:
      - Only precedence/activity edges E contribute to duration
      - Policy edges enforce ordering but contribute zero duration
    """
    edges_all = list(set(edges_prec) | set(edges_policy))
    n = len(bar_p)

    model = Model("robust_nominal")
    if not gurobi_output:
        model.Params.OutputFlag = 0

    # phi over all edges
    phi = {}
    for (i, j) in edges_all:
        phi[(i, j)] = model.addVar(vtype=GRB.BINARY, name=f"phi_{i}_{j}")
    model.update()

    # Objective: weights only on activity arcs (E)
    obj = quicksum(-bar_p[i] * phi[(i, j)] for (i, j) in edges_prec)
    model.setObjective(obj, GRB.MINIMIZE)

    nodes = list(range(n))

    # Flow constraints
    for i in nodes:
        outgoing = quicksum(phi[(i, j)] for (i, j) in edges_all if i == i)
        incoming = quicksum(phi[(j, i)] for (j, i) in edges_all if i == i)

        if i == source:
            model.addConstr(outgoing == 1)
        elif i == sink:
            model.addConstr(incoming == 1)
        else:
            model.addConstr(outgoing - incoming == 0)

    model.optimize()

    if model.Status != GRB.OPTIMAL:
        raise RuntimeError("Longest-path (adversary) not optimal")

    adv_obj = model.ObjVal
    duration = -adv_obj
    phi_values = {(i, j): float(phi[(i, j)].X) for (i, j) in edges_all}

    return duration, adv_obj, phi_values



def build_worst_case_durations_simple(active_nodes, bar_p, hat_p, Gamma):
    """
    Allocate BS-budget Gamma across active nodes:
       u_i ∈ [0,1], sum u_i ≤ Gamma
    """
    n = len(bar_p)
    p_star = [float(bar_p[i]) for i in range(n)]

    sorted_nodes = sorted(active_nodes, key=lambda i: hat_p[i], reverse=True)

    Gamma_left = float(Gamma)
    for i in sorted_nodes:
        if Gamma_left <= 0:
            break
        u_i = min(1.0, Gamma_left)
        p_star[i] = bar_p[i] + hat_p[i] * u_i
        Gamma_left -= u_i

    return p_star



def solve_adversary_over_all_pi_simple(bar_p, hat_p, E, policy_edges, source, sink, Gamma, gurobi_output=False):
    """
    Correct adversary for BS-budget:
      1. Nominal longest path => φ_nom => active nodes
      2. Allocate Gamma across active nodes => build p_star
      3. Longest path under p_star => robust duration
    """

    # Step 1: nominal longest path (E only)
    nominal_dur, _, phi_nom = solve_nominal_with_gurobi_simple(
        bar_p=bar_p,
        edges_prec=E,
        edges_policy=policy_edges,
        source=source,
        sink=sink,
        gurobi_output=gurobi_output
    )

    # Step 2: extract active nodes
    active_nodes = get_active_nodes_from_phi_simple(
        phi_values=phi_nom,
        n=len(bar_p),
        source=source,
        sink=sink,
    )

    # Step 3: allocate Gamma (BS-budget)
    p_star = build_worst_case_durations_simple(
        active_nodes=active_nodes,
        bar_p=bar_p,
        hat_p=hat_p,
        Gamma=Gamma,
    )

    # Step 4: compute worst-case longest path
    robust_dur, robust_adv, phi_rob = solve_nominal_with_gurobi_simple(
        bar_p=p_star,
        edges_prec=E,
        edges_policy=policy_edges,
        source=source,
        sink=sink,
        gurobi_output=gurobi_output
    )

    return None, robust_dur, robust_adv, phi_rob, active_nodes, p_star, []


def get_active_nodes_from_phi_simple(phi_values, n, source, sink, tol=1e-6):
    """
    n: number of nodes (nodes are 0,...,n-1)
    phi_values: dict {(i,j): value}
    returns list of nodes i (excluding source,sink) with sum_j phi_ij ≈ 1
    """
    outgoing_sum = [0.0] * n
    for (i, j), val in phi_values.items():
        outgoing_sum[i] += float(val)

    active_nodes = [
        i for i in range(n)
        if i not in (source, sink) and abs(outgoing_sum[i] - 1.0) <= tol
    ]
    return active_nodes


# ===============================================================
#   Correct Robust Adversary for BS-budget RCPSP
#   (handles precedence arcs separately from policy arcs)
# ===============================================================


def solve_nominal_with_gurobi_simple(bar_p, edges_prec, edges_policy, source, sink, gurobi_output=False):
    """
    Compute longest path where:
      - Only precedence/activity edges E contribute to duration
      - Policy edges enforce ordering but contribute zero duration
    """
    edges_all = list(set(edges_prec) | set(edges_policy))
    n = len(bar_p)

    model = Model("robust_nominal")
    if not gurobi_output:
        model.Params.OutputFlag = 0

    # phi over all edges
    phi = {}
    for (i, j) in edges_all:
        phi[(i, j)] = model.addVar(vtype=GRB.BINARY, name=f"phi_{i}_{j}")
    model.update()

    # Objective: weights only on activity arcs (E)
    obj = quicksum(-bar_p[i] * phi[(i, j)] for (i, j) in edges_prec)
    model.setObjective(obj, GRB.MINIMIZE)

    nodes = list(range(n))

    # Flow constraints
    for i in nodes:
        outgoing = quicksum(phi[(i, j)] for (i, j) in edges_all if i == i)
        incoming = quicksum(phi[(j, i)] for (j, i) in edges_all if i == i)

        if i == source:
            model.addConstr(outgoing == 1)
        elif i == sink:
            model.addConstr(incoming == 1)
        else:
            model.addConstr(outgoing - incoming == 0)

    model.optimize()

    if model.Status != GRB.OPTIMAL:
        raise RuntimeError("Longest-path (adversary) not optimal")

    adv_obj = model.ObjVal
    duration = -adv_obj
    phi_values = {(i, j): float(phi[(i, j)].X) for (i, j) in edges_all}

    return duration, adv_obj, phi_values



def build_worst_case_durations_simple(active_nodes, bar_p, hat_p, Gamma):
    """
    Allocate BS-budget Gamma across active nodes:
       u_i ∈ [0,1], sum u_i ≤ Gamma
    """
    n = len(bar_p)
    p_star = [float(bar_p[i]) for i in range(n)]

    sorted_nodes = sorted(active_nodes, key=lambda i: hat_p[i], reverse=True)

    Gamma_left = float(Gamma)
    for i in sorted_nodes:
        if Gamma_left <= 0:
            break
        u_i = min(1.0, Gamma_left)
        p_star[i] = bar_p[i] + hat_p[i] * u_i
        Gamma_left -= u_i

    return p_star



def solve_adversary_over_all_pi_simple(bar_p, hat_p, E, policy_edges, source, sink, Gamma, gurobi_output=False):
    """
    Correct adversary for BS-budget:
      1. Nominal longest path => φ_nom => active nodes
      2. Allocate Gamma across active nodes => build p_star
      3. Longest path under p_star => robust duration
    """

    # Step 1: nominal longest path (E only)
    nominal_dur, _, phi_nom = solve_nominal_with_gurobi_simple(
        bar_p=bar_p,
        edges_prec=E,
        edges_policy=policy_edges,
        source=source,
        sink=sink,
        gurobi_output=gurobi_output
    )

    # Step 2: extract active nodes
    active_nodes = get_active_nodes_from_phi_simple(
        phi_values=phi_nom,
        n=len(bar_p),
        source=source,
        sink=sink,
    )

    # Step 3: allocate Gamma (BS-budget)
    p_star = build_worst_case_durations_simple(
        active_nodes=active_nodes,
        bar_p=bar_p,
        hat_p=hat_p,
        Gamma=Gamma,
    )

    # Step 4: compute worst-case longest path
    robust_dur, robust_adv, phi_rob = solve_nominal_with_gurobi_simple(
        bar_p=p_star,
        edges_prec=E,
        edges_policy=policy_edges,
        source=source,
        sink=sink,
        gurobi_output=gurobi_output
    )

    return None, robust_dur, robust_adv, phi_rob, active_nodes, p_star, []



In [128]:
def worst_case_rcpsp_meta(
    V,
    E,
    R,
    bar_p,
    hat_p,
    Gamma,
    b_node,
    b_supply,
    epsilon=1e-3,
    M_prec=1e6,
    M_flow=1e6,
    max_iters=20,
    gurobi_output=False,
):
    """
    Meta-algorithm for the Worst-Case RCPSP under budgeted duration uncertainty.

    Implements Algorithm 'Worst-Case RCPSP under Budgeted Duration Uncertainty':

      Initialization:
        - choose initial duration vector p^1 in P (here: p^1 = bar_p)
        - P_hat_1 = {p^1}
        - q = 1, LB = 0, UB = +inf

      While UB - LB > epsilon:
        Step 1: Restricted Master over P_hat_q -> X^q, LB
        Step 2: Adversary -> rho_max(X^q), worst-case durations p^{q+1}
        Step 3: Update UB, and if LB != UB add new scenario and continue

    Inputs
    ------
    V        : list of nodes (0 is source, V[-1] is sink)
    E        : list of fixed precedence arcs (i,j)
    R        : list of resource indices
    bar_p    : list of nominal durations (length |V|)
    hat_p    : list of deviations (length |V|)
    Gamma    : uncertainty budget
    b_node   : node-by-resource balances b_{ik} (dict[(i,k)] or 2D list [i][k])
    b_supply : per-resource supply b_k (dict[k] or list)
    epsilon  : tolerance for stopping condition on UB - LB
    M_prec   : big-M for precedence constraints
    M_flow   : big-M for flow capacity constraints
    max_iters: safety cap on number of iterations
    gurobi_output: whether to print Gurobi logs

    Returns
    -------
    best_policy : list of arcs (i,j) representing X^*
    rho_star    : worst-case makespan (approx. = UB at termination)
    LB_hist     : list of LB values per iteration
    UB_hist     : list of UB values per iteration
    scenarios   : list of all scenario vectors p^h that were added
    """
    source = V[0]
    sink   = V[-1]

    # Initial scenario p^1: we take the nominal vector bar_p
    p1 = list(bar_p)
    P_list = [p1]

    # Build initial master over P_hat_1 = {p^1}
    m, x, S, rho, f = create_master_problem(
        V=V,
        E=E,
        P_list=P_list,
        R=R,
        b_node=b_node,
        b_supply=b_supply,
        M_prec=M_prec,
        M_flow=M_flow,
        gurobi_output=gurobi_output,
    )

    q = 1
    LB = 0.0
    UB = math.inf

    LB_hist = []
    UB_hist = []

    best_policy = None
    best_UB = UB

    for it in range(1, max_iters + 1):
        # ---- Step 1: solve restricted master over current P_hat_q ----
        m.optimize()
        if m.Status != 2:  # GRB.OPTIMAL
            raise RuntimeError(f"Master problem not optimal, status={m.Status}")

        LB = float(rho.X)
        LB_hist.append(LB)

        # Extract current earliest-start policy X^q from x
        policy_edges = extract_policy_edges_from_x(x, V, tol=0.5)
        print(f"Iter {it}: selection arcs (policy X^{q}) = {policy_edges}")  # <-- ADDED

        # Adversary graph uses base arcs E and chosen policy arcs X^q
        adversary_edges = list(set(E) | set(policy_edges))

        # ---- Step 2: adversarial worst-case for X^q ----
        best_pi, best_duration, best_adv_obj, best_phi, active_nodes, p_star, pi_values = \
            solve_adversary_over_all_pi_simple(
                bar_p=bar_p,
                hat_p=hat_p,
                edges=adversary_edges,
                source=source,
                sink=sink,
                Gamma=Gamma,
                gurobi_output=gurobi_output,
            )
        print(best_pi)
        rho_max_Xq = float(best_duration)
        UB = min(UB, rho_max_Xq)
        UB_hist.append(UB)

        # Track best policy and UB so far
        if rho_max_Xq <= best_UB:
            best_UB = rho_max_Xq
            best_policy = list(policy_edges)

        # Stopping criterion
        if UB - LB <= epsilon:
            break

        # ---- Step 3: update scenario set if LB != UB ----
        # Add new scenario p^{q+1} = p_star to the master as h_new
        q += 1
        h_new = q - 1
        P_list.append(list(p_star))

        add_scenario_to_master(
            m=m,
            x=x,
            S=S,
            rho=rho,
            p_vec=p_star,
            h_new=h_new,
            V=V,
            M_prec=M_prec,
        )

    # At termination, best_policy is our X^*, best_UB approximates rho^*
    rho_star = best_UB

    return best_policy, rho_star, LB_hist, UB_hist, P_list


**EXAMPLES**

In [129]:
import math

# Tiny toy instance
V = [0, 1, 2, 3]                 # 0 = source, 3 = sink
E = [(0, 1), (1, 2), (2, 3)]     # fixed precedence path

# One resource
R = [0]

# Nominal durations bar_p and deviations hat_p
bar_p = [0.0, 3.0, 2.0, 0.0]     # durations for nodes 0..3
hat_p = [0.0, 1.0, 0.5, 0.0]     # deviations for nodes 0..3

# Uncertainty budget Gamma
Gamma = 1.5

# Flow balances b_{ik} (here as list-of-lists [i][k])
# internal nodes (1 and 2) have 1 unit in and out; source/sink have 0
b_node = [
    [0.0],   # i = 0 (source)
    [1.0],   # i = 1 (internal)
    [1.0],   # i = 2 (internal)
    [0.0],   # i = 3 (sink)
]

# Total supply b_k from source to sink for resource k=0
b_supply = [1.0]


In [130]:
best_policy, rho_star, LB_hist, UB_hist, scenarios = worst_case_rcpsp_meta(
    V=V,
    E=E,
    R=R,
    bar_p=bar_p,
    hat_p=hat_p,
    Gamma=Gamma,
    b_node=b_node,
    b_supply=b_supply,
    epsilon=1e-3,
    M_prec=1e4,
    M_flow=1e4,
    max_iters=10,
    gurobi_output=False,   
)

print("=== FINAL RESULT ===")
print("Best policy X* (edges):", best_policy)
print("Worst-case makespan rho*:", rho_star)
print("Scenarios used (P_hat_q):")
for h, p in enumerate(scenarios):
    print(f"  h={h}: p^{h+1} = {p}")

print("\nLB / UB history:")
for t, (lb, ub) in enumerate(zip(LB_hist, UB_hist), start=1):
    print(f"  iter {t}: LB={lb:.4f}, UB={ub:.4f}, gap={ub - lb:.4f}")


Iter 1: selection arcs (policy X^1) = [(0, 1), (1, 2), (2, 3)]


TypeError: solve_adversary_over_all_pi_simple() got an unexpected keyword argument 'edges'

In [131]:
# Example data: 6 nodes (0..5), 2 resources, less degenerate edge set

V = [0, 1, 2, 3, 4, 5]   # 0 = source, 5 = sink
source = 0
sink = 5

# Less degenerate precedence arcs: branching & merging
# 0 → 1 → 3 → 5
#  \        ↑
#   → 2 → 4 ┘
#   \→ 3
E = [
    (0, 1),
    (0, 2),
    (1, 3),
    (2, 3),
    (2, 4),
    (3, 5),
    (4, 5),
]

# Nominal durations and deviations
bar_p = [0.0, 3.0, 2.5, 4.0, 1.5, 0.0]
hat_p = [0.0, 1.0, 0.8, 0.5, 0.7, 0.0]

Gamma = 2.2

# Two resources
R = [0, 1]

# Node-by-resource balances b_{ik}
b_node = [
    [0.0, 0.0],   # node 0 (source)
    [1.0, 2.0],   # node 1
    [1.0, 2.0],   # node 2
    [0.0, 1.0],   # node 3
    [1.0, 1.0],   # node 4
    [0.0, 0.0],   # node 5 (sink)
]

# Total supply per resource
b_supply = [1.0, 2.0]

# Run the meta-algorithm (assumes all functions are already defined)
best_policy, rho_star, LB_hist, UB_hist, scenarios = worst_case_rcpsp_meta(
    V=V,
    E=E,
    R=R,
    bar_p=bar_p,
    hat_p=hat_p,
    Gamma=Gamma,
    b_node=b_node,
    b_supply=b_supply,
    epsilon=1e-3,
    M_prec=1e4,
    M_flow=1e4,
    max_iters=10,
    gurobi_output=False,
)

print("=== FINAL RESULT ===")
print("Best policy X* (edges):", best_policy)
print("Worst-case makespan rho*:", rho_star)

print("\nScenarios used (P_hat_q):")
for h, p in enumerate(scenarios):
    print(f"  h={h}: p^{h+1} = {p}")

print("\nLB / UB history:")
for t, (lb, ub) in enumerate(zip(LB_hist, UB_hist), start=1):
    print(f"  iter {t}: LB={lb:.4f}, UB={ub:.4f}, gap={ub - lb:.4f}")



p

Iter 1: selection arcs (policy X^1) = [(0, 1), (0, 2), (0, 5), (1, 2), (1, 3), (1, 5), (2, 3), (2, 4), (2, 5), (3, 5), (4, 5)]


TypeError: solve_adversary_over_all_pi_simple() got an unexpected keyword argument 'edges'